In [ ]:
import React, { useState, useEffect } from 'react';
import { LineChart, Line, BarChart, Bar, XAxis, YAxis, CartesianGrid, Tooltip, Legend, ResponsiveContainer } from 'recharts';
import Papa from 'papaparse';
import _ from 'lodash';

const MadridTransportDashboard = () => {
  const [data, setData] = useState([]);
  const [weekdayAverages, setWeekdayAverages] = useState([]);
  const [monthlyAverages, setMonthlyAverages] = useState([]);
  const [correlationData, setCorrelationData] = useState([]);
  const [loading, setLoading] = useState(true);

  useEffect(() => {
    const processData = async () => {
      try {
        setLoading(true);
        const response = await window.fs.readFile('paste.txt');
        const text = new TextDecoder().decode(response);
        
        // Parsear el CSV
        const parsedData = Papa.parse(text, {
          delimiter: "\t",
          header: true,
          skipEmptyLines: true,
          transformHeader: header => header.trim()
        });
        
        // Función para procesar números - corrigiendo formato con comas como separadores de miles
        const cleanNumber = (value) => {
          if (typeof value === 'string') {
            // En el formato español, las comas son separadores decimales y puntos son separadores de miles
            return parseFloat(value.replace(/\./g, '').replace(/,/g, '.'));
          }
          return value;
        };
        
        // Función para determinar el tipo de día
        const getDayType = (dateStr) => {
          const parts = dateStr.split('-');
          if (parts.length !== 3) return 'desconocido';
          
          const date = new Date(20 + parts[2], parts[1] - 1, parts[0]);
          const dayOfWeek = date.getDay();
          
          const holidays = [
            "1-1-23", "6-1-23", "7-4-23", "1-5-23", "15-8-23", "12-10-23", "1-11-23", "6-12-23", "8-12-23", "25-12-23",
            "1-1-24", "6-1-24", "29-3-24", "1-5-24", "15-8-24", "12-10-24", "1-11-24", "6-12-24", "25-12-24",
            "1-1-25", "6-1-25"
          ];
          
          if (holidays.includes(dateStr)) return 'festivo';
          if (dayOfWeek === 0) return 'domingo';
          if (dayOfWeek === 6) return 'sábado';
          return 'laborable';
        };
        
        // Procesar datos
        const transformedData = parsedData.data
          .filter(row => row[''] && row['Metro de Madrid'])
          .map(row => {
            const date = row[''];
            return {
              date,
              dayOfWeek: new Date(date.split('-').reverse().map((p, i) => i === 0 ? '20' + p : p).join('-')).getDay(),
              dayType: getDayType(date),
              month: parseInt(date.split('-')[1]),
              year: parseInt('20' + date.split('-')[2]),
              metro: cleanNumber(row['Metro de Madrid']),
              emt: cleanNumber(row['EMT']),
              carretera: cleanNumber(row['Conc. por carretera']),
              renfe: cleanNumber(row['Renfe Cercanías']),
              total: cleanNumber(row['Total'])
            };
          });
        
        // Ordenar por fecha
        transformedData.sort((a, b) => {
          const dateA = new Date(a.date.split('-').reverse().map((p, i) => i === 0 ? '20' + p : p).join('-'));
          const dateB = new Date(b.date.split('-').reverse().map((p, i) => i === 0 ? '20' + p : p).join('-'));
          return dateA - dateB;
        });
        
        setData(transformedData);
        
        // Calcular promedios por día de la semana
        const dayNames = ['Domingo', 'Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado'];
        const weekdayData = _.range(0, 7).map(dayNum => {
          const dayData = transformedData.filter(item => item.dayOfWeek === dayNum);
          return {
            name: dayNames[dayNum],
            metro: _.meanBy(dayData, 'metro').toFixed(2),
            emt: _.meanBy(dayData, 'emt').toFixed(2),
            carretera: _.meanBy(dayData, 'carretera').toFixed(2),
            renfe: _.meanBy(dayData, 'renfe').toFixed(2)
          };
        });
        setWeekdayAverages(weekdayData);
        
        // Calcular promedios mensuales
        const monthNames = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'];
        const monthlyData = _.range(1, 13).map(month => {
          const monthData = transformedData.filter(item => item.month === month);
          return {
            name: monthNames[month - 1],
            metro: _.meanBy(monthData, 'metro').toFixed(2),
            emt: _.meanBy(monthData, 'emt').toFixed(2),
            carretera: _.meanBy(monthData, 'carretera').toFixed(2),
            renfe: _.meanBy(monthData, 'renfe').toFixed(2)
          };
        });
        setMonthlyAverages(monthlyData);
        
        // Calcular correlaciones
        const transportModes = ['metro', 'emt', 'carretera', 'renfe'];
        const correlations = [];
        for (let i = 0; i < transportModes.length; i++) {
          for (let j = i + 1; j < transportModes.length; j++) {
            const mode1 = transportModes[i];
            const mode2 = transportModes[j];
            
            // Calcular correlación simple
            const n = transformedData.length;
            const xMean = _.meanBy(transformedData, mode1);
            const yMean = _.meanBy(transformedData, mode2);
            
            let covariance = 0;
            let xVar = 0;
            let yVar = 0;
            
            for (const row of transformedData) {
              const xDiff = row[mode1] - xMean;
              const yDiff = row[mode2] - yMean;
              
              covariance += xDiff * yDiff;
              xVar += xDiff * xDiff;
              yVar += yDiff * yDiff;
            }
            
            covariance /= n;
            xVar /= n;
            yVar /= n;
            
            const correlation = covariance / (Math.sqrt(xVar) * Math.sqrt(yVar));
            
            correlations.push({
              pair: `${mode1} - ${mode2}`,
              value: correlation.toFixed(4)
            });
          }
        }
        setCorrelationData(correlations);
        
        setLoading(false);
      } catch (error) {
        console.error('Error processing data:', error);
        setLoading(false);
      }
    };
    
    processData();
  }, []);
  
  if (loading) {
    return <div className="p-8 text-center">Cargando datos...</div>;
  }
  
  // Preparar datos para la visualización de tendencias
  const timeSeriesData = data.map(item => ({
    date: item.date,
    metro: item.metro,
    emt: item.emt,
    carretera: item.carretera,
    renfe: item.renfe
  }));
  
  // Datos para promedios por tipo de día
  const dayTypeData = [
    { name: 'Laborable', metro: 0, emt: 0, carretera: 0, renfe: 0, count: 0 },
    { name: 'Sábado', metro: 0, emt: 0, carretera: 0, renfe: 0, count: 0 },
    { name: 'Domingo', metro: 0, emt: 0, carretera: 0, renfe: 0, count: 0 },
    { name: 'Festivo', metro: 0, emt: 0, carretera: 0, renfe: 0, count: 0 }
  ];
  
  // Calcular sumas para cada tipo de día
  data.forEach(item => {
    let index;
    if (item.dayType === 'laborable') index = 0;
    else if (item.dayType === 'sábado') index = 1;
    else if (item.dayType === 'domingo') index = 2;
    else if (item.dayType === 'festivo') index = 3;
    else return;
    
    dayTypeData[index].metro += item.metro;
    dayTypeData[index].emt += item.emt;
    dayTypeData[index].carretera += item.carretera;
    dayTypeData[index].renfe += item.renfe;
    dayTypeData[index].count += 1;
  });
  
  // Calcular promedios
  dayTypeData.forEach(item => {
    if (item.count > 0) {
      item.metro = (item.metro / item.count).toFixed(2);
      item.emt = (item.emt / item.count).toFixed(2);
      item.carretera = (item.carretera / item.count).toFixed(2);
      item.renfe = (item.renfe / item.count).toFixed(2);
    }
    delete item.count;
  });
  
  return (
    <div className="p-4 bg-gray-50 min-h-screen">
      <h1 className="text-2xl font-bold mb-6">Análisis de Transporte en Madrid</h1>
      
      <div className="mb-8 bg-white p-4 rounded-lg shadow">
        <h2 className="text-xl font-semibold mb-4">Distribución por Tipo de Día</h2>
        <ResponsiveContainer width="100%" height={300}>
          <BarChart data={dayTypeData}>
            <CartesianGrid strokeDasharray="3 3" />
            <XAxis dataKey="name" />
            <YAxis tickFormatter={(value) => `${(value / 1000000).toFixed(1)}M`} />
            <Tooltip formatter={(value) => [`${Number(value).toLocaleString()} viajeros`, '']} />
            <Legend />
            <Bar dataKey="metro" name="Metro" fill="#8884d8" />
            <Bar dataKey="emt" name="EMT" fill="#82ca9d" />
            <Bar dataKey="carretera" name="Concesiones" fill="#ffc658" />
            <Bar dataKey="renfe" name="Cercanías" fill="#ff8042" />
          </BarChart>
        </ResponsiveContainer>
        <p className="mt-2 text-sm text-gray-600">Número promedio de viajeros por tipo de día</p>
      </div>
      
      <div className="mb-8 bg-white p-4 rounded-lg shadow">
        <h2 className="text-xl font-semibold mb-4">Promedios por Día de la Semana</h2>
        <ResponsiveContainer width="100%" height={300}>
          <BarChart data={weekdayAverages}>
            <CartesianGrid strokeDasharray="3 3" />
            <XAxis dataKey="name" />
            <YAxis tickFormatter={(value) => `${(value / 1000000).toFixed(1)}M`} />
            <Tooltip formatter={(value) => [`${Number(value).toLocaleString()} viajeros`, '']} />
            <Legend />
            <Bar dataKey="metro" name="Metro" fill="#8884d8" />
            <Bar dataKey="emt" name="EMT" fill="#82ca9d" />
            <Bar dataKey="carretera" name="Concesiones" fill="#ffc658" />
            <Bar dataKey="renfe" name="Cercanías" fill="#ff8042" />
          </BarChart>
        </ResponsiveContainer>
        <p className="mt-2 text-sm text-gray-600">Número promedio de viajeros por día de la semana</p>
      </div>
      
      <div className="mb-8 bg-white p-4 rounded-lg shadow">
        <h2 className="text-xl font-semibold mb-4">Estacionalidad Mensual</h2>
        <ResponsiveContainer width="100%" height={300}>
          <LineChart data={monthlyAverages}>
            <CartesianGrid strokeDasharray="3 3" />
            <XAxis dataKey="name" />
            <YAxis tickFormatter={(value) => `${(value / 1000000).toFixed(1)}M`} />
            <Tooltip formatter={(value) => [`${Number(value).toLocaleString()} viajeros`, '']} />
            <Legend />
            <Line type="monotone" dataKey="metro" name="Metro" stroke="#8884d8" />
            <Line type="monotone" dataKey="emt" name="EMT" stroke="#82ca9d" />
            <Line type="monotone" dataKey="carretera" name="Concesiones" stroke="#ffc658" />
            <Line type="monotone" dataKey="renfe" name="Cercanías" stroke="#ff8042" />
          </LineChart>
        </ResponsiveContainer>
        <p className="mt-2 text-sm text-gray-600">Variación de viajeros promedio por mes del año</p>
      </div>
      
      <div className="mb-8 bg-white p-4 rounded-lg shadow">
        <h2 className="text-xl font-semibold mb-4">Correlaciones entre Medios de Transporte</h2>
        <ResponsiveContainer width="100%" height={300}>
          <BarChart data={correlationData} layout="vertical">
            <CartesianGrid strokeDasharray="3 3" />
            <XAxis type="number" domain={[0, 1]} />
            <YAxis dataKey="pair" type="category" width={150} />
            <Tooltip formatter={(value) => [value, 'Correlación']} />
            <Bar dataKey="value" name="Correlación" fill="#8884d8" />
          </BarChart>
        </ResponsiveContainer>
        <p className="mt-4 text-gray-600 text-sm">
          Todas las correlaciones son muy altas (>0.95), lo que sugiere que los diferentes modos de transporte 
          siguen patrones muy similares. Esto puede ser útil para los modelos multivariables.
        </p>
      </div>
      
      <div className="mt-8 p-6 bg-white rounded-lg shadow">
        <h2 className="text-xl font-semibold mb-4">Propuesta de Modelos para Series Temporales</h2>
        
        <div className="mb-6">
          <h3 className="text-lg font-medium mb-2">1. Enfoque Ingenuo (Baseline)</h3>
          <p>Variables de entrada: Valores históricos del medio de transporte + tipo de día + mes</p>
          <p>Métodos: Media móvil, suavizado exponencial, naive seasonal (mismo día de la semana anterior)</p>
          <p className="text-gray-600 mt-1">Este enfoque sirve como baseline y puede capturar las claras diferencias entre días (2.1M viajeros en día laborable vs 1.1M en festivos para Metro).</p>
        </div>
        
        <div className="mb-6">
          <h3 className="text-lg font-medium mb-2">2. Modelos Lineales</h3>
          <p>Variables: Datos históricos + variables categóricas para día/mes + interacciones entre medios</p>
          <p>Métodos: Regresión múltiple, VAR (Vector Autoregression)</p>
          <p className="text-gray-600 mt-1">Las correlaciones son extremadamente altas entre los diferentes modos de transporte (>0.96), lo que hace al VAR especialmente adecuado.</p>
        </div>
        
        <div className="mb-6">
          <h3 className="text-lg font-medium mb-2">3. Redes Neuronales Simples</h3>
          <p>Estructura: Redes feedforward con 1-2 capas ocultas</p>
          <p>Entradas: Valores históricos, variables categóricas codificadas (tipo de día y mes)</p>
          <p className="text-gray-600 mt-1">Con volúmenes que oscilan entre 700K y 2.7M de viajeros diarios en Metro, estas redes pueden capturar patrones no lineales.</p>
        </div>
        
        <div className="mb-6">
          <h3 className="text-lg font-medium mb-2">4. Redes Neuronales Profundas</h3>
          <p>Arquitecturas: LSTM, GRU o Transformers con múltiples capas</p>
          <p>Enfoque: Secuencias temporales multivariable con todos los medios como entradas</p>
          <p className="text-gray-600 mt-1">Especialmente útiles para modelar la fuerte estacionalidad (mensual y semanal) que muestran los datos, con la clara caída de agosto.</p>
        </div>
        
        <div className="mb-6">
          <h3 className="text-lg font-medium mb-2">5. Modelos ARIMA/SARIMA</h3>
          <p>Enfoque: VARIMA (Vector ARIMA) para modelado multivariable</p>
          <p>Componentes: Autorregresivo, integrado y de media móvil con componente estacional</p>
          <p className="text-gray-600 mt-1">Los datos muestran patrones estacionales claros tanto semanales como anuales que VARIMA puede modelar explícitamente.</p>
        </div>
        
        <div className="mt-8">
          <h3 className="text-lg font-medium mb-2">Consideraciones adicionales:</h3>
          <ul className="list-disc pl-6">
            <li>Todos los modelos deben incorporar la estacionalidad diaria y mensual observada</li>
            <li>Las variables categóricas (tipo de día) son críticas para todos los modelos</li>
            <li>Se recomienda crear modelos para días laborables y fines de semana/festivos por separado</li>
            <li>Evaluación cruzada: usar validación temporal (no aleatorizada)</li>
            <li>Métricas: RMSE, MAE y MAPE para comparar todos los modelos</li>
          </ul>
        </div>
      </div>
    </div>
  );
};

export default MadridTransportDashboard;